<a href="https://colab.research.google.com/github/Matheus-Fuzati-de-Carvalho/Matheus-Fuzati-de-Carvalho/blob/main/Monitoramento_Inscri%C3%A7%C3%B5es_e_Matr%C3%ADculas_25_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando bibliotecas
from google.cloud import bigquery
import pandas as pd
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe
gc = gspread.authorize(creds)

# Trazendo a data atual para a váriavel 'Atualizado'
from datetime import datetime, timedelta
import pytz
atualizado = pytz.timezone('America/Sao_Paulo')
atualizado = datetime.now(atualizado).strftime("%d/%m/%Y, %H:%M:%S")

hoje =  datetime.now().strftime("%Y-%m-%d")

# Desativando avisos sobre atribuições encadeadas
pd.options.mode.chained_assignment = None


In [157]:
# definindo função para abrir bases
def abrir_plan(arq, plan):
  planilha = gc.open(arq)
  planilha = planilha.get_worksheet(plan).get_all_values()
  planilha = pd.DataFrame(planilha)
  planilha.columns = planilha.iloc[0]
  planilha = planilha[1:].reset_index(drop=True)
  return planilha

# Criando função para atualizar a planilha
def att_plan(arquivo, planilha, dataframe):
  arq = gc.open(arquivo)
  plan = arq.worksheet(planilha)
  set_with_dataframe(plan, dataframe)

In [158]:
# Abrindo base Rubeus referente a 20251
base25_script = abrir_plan('base_rubeus_script', 0)

In [159]:
# base25_script.info()

In [160]:
# base25_script[(base25_script['etapaNome'].isin(['Matriculado', 'Matrícula Cancelada'])) & (base25_script['processoNome'] == 'Inscrições e Matrículas')]['pessoaNome'].value_counts()

In [161]:
# Abrindo base de inscritos de 241
base24 = abrir_plan('base241', 0)

# Cria uma cópia de 'base25_script' para evitar alterar o DataFrame original
base25 = base25_script.copy()

# Abrir planilha de auxilio a posições
base_aux = abrir_plan('base241', 1)

# Abrir planilha de emails TRV
base_trv = abrir_plan('RESERVA DE MATRÍCULA 2025.1', 0)

In [162]:
# base25.info()

In [163]:
# Cria uma função para verificar se uma célula contém a palavra "teste"
def contem_teste(coluna):
    return coluna.str.contains('teste', case=False, na=False)

# Aplica as funções a todas as colunas e cria uma máscara booleana
mascara_teste = base25.apply(contem_teste).any(axis=1)

# Exclui as linhas que contêm "teste"
base25 = base25[~(mascara_teste)]

In [164]:
base25['temperatura'] = base25['temperatura'].astype('object')
base25['temperatura'] = base25['temperatura'].str.replace('["', '')
base25['temperatura'] = base25['temperatura'].str.replace('"]', '')

In [165]:
print(len(base25[(base25['tags'].str.contains('#preinscricao-VestibularDesafio-15feteps', na=False)) &  # Verifica se a string está em 'tags'
    (base25['anoDeEstudoETEC'].isin(['3º Ano', 'Concluiu o ensino médio'])) &  # Verifica o ano de estudo
    (base25['processoNome'] == 'Comunicação e Marketing')]))

print(len(base25[(base25['tags'].str.contains('#preinscricao-VestibularDesafio2025'))
                            & (base25['processoNome'] == 'Comunicação e Marketing')]))


print(len(base25[base25['processoNome'] == 'Inscrições e Matrículas']))

781
63
3481


In [166]:
# Filtra o DataFrame base25 para manter apenas os registros relacionados aos processos 'Inscrições e Matrículas' e 'Comunicação e Marketing'
base25 = base25[base25['processoNome'].isin(['Inscrições e Matrículas', 'Comunicação e Marketing'])]

# Cria uma nova base contendo apenas os registros com a tag '#preinscricao-VestibularDesafio-15feteps',
# que correspondem a alunos do 3º ano ou que já concluíram o ensino médio, e que estão no processo 'Comunicação e Marketing'
basePreInscricaoVestibularFeteps = (base25[(base25['tags'].str.contains('#preinscricao-VestibularDesafio-15feteps', na=False))
                                 & (base25['anoDeEstudoETEC'].isin(['3º Ano', 'Concluiu o ensino médio']))
                                 & (base25['processoNome'] == 'Comunicação e Marketing')])

# Define novas colunas para essa base: 'baseDeOrigem', 'etapaNome' e 'processoSeletivoNome'
# Essas colunas indicam que esses registros são pré-inscritos no Vestibular FETEPS, na etapa de pré-inscrição e sem processo definido
basePreInscricaoVestibularFeteps['baseDeOrigem'] = 'PRÉ-INSCRITO VESTIBULAR - FETEPS'
basePreInscricaoVestibularFeteps['etapaNome'] = 'Pré-Inscrito'
basePreInscricaoVestibularFeteps['processoSeletivoNome'] = 'Sem processo definido'

# Cria uma nova base com registros que contêm a tag '#preinscricao-VestibularDesafio2025' e pertencem ao processo 'Comunicação e Marketing'
basePreInscritoVestibular = (base25[(base25['tags'].str.contains('#preinscricao-VestibularDesafio2025'))
                            & (base25['processoNome'] == 'Comunicação e Marketing')])

# Define novas colunas para essa base: 'baseDeOrigem', 'etapaNome' e 'processoSeletivoNome'
# Indicando que esses registros são pré-inscritos no Vestibular de 2025, na etapa de pré-inscrição e fazem parte do processo seletivo '25.1 - Desafio ESEG'
basePreInscritoVestibular['baseDeOrigem'] = 'PRÉ-INSCRITO VESTIBULAR - SITE'
basePreInscritoVestibular['etapaNome'] = 'Pré-Inscrito'
basePreInscritoVestibular['processoSeletivoNome'] = '25.1 - Desafio ESEG'


In [167]:
base25[base25['etapaNome'].isin(['Matriculado', 'Matrícula Cancelada'])]['pessoaNome'].value_counts().sort_values( ascending=True).sum()

102

In [168]:
# Abre a planilha 'base241' na segunda aba e atribui ao DataFrame fonteInscricao
fonteInscricao = abrir_plan('base241', 2)

# Ordena o DataFrame 'fonteInscricao' priorizando os registros onde 'baseDeOrigem' é 'Pré Inscritos FETEPS'
# Dessa forma, ao remover duplicados, o registro com 'Pré Inscritos FETEPS' será mantido, se houver duplicatas no campo 'email'
fonteInscricao = fonteInscricao.sort_values(by=['email', 'baseDeOrigem'], key=lambda col: col == 'Pré Inscritos FETEPS', ascending=False)

# Remove registros duplicados com base na coluna 'email', mantendo o primeiro (que é priorizado para ser da origem 'Pré Inscritos FETEPS')
fonteInscricao = fonteInscricao.drop_duplicates(subset='email', keep='first')

# Filtra o DataFrame 'base25' para incluir apenas registros do processo 'Inscrições e Matrículas'
base25 = base25[base25['processoNome'] == 'Inscrições e Matrículas']

# Realiza o merge de 'base25' com 'fonteInscricao' na coluna 'email', fazendo uma junção à esquerda ('left')
# Isso adiciona as informações de 'fonteInscricao' aos registros de 'base25' com base no campo 'email'
base25 = pd.merge(base25, fonteInscricao, on='email', how='left')

# Para os registros em que a coluna 'baseDeOrigem' está vazia (NaN), atribui o valor 'SITE' para indicar a origem
base25.loc[base25['baseDeOrigem'].isna(), 'baseDeOrigem'] = 'SITE'

# Concatena três DataFrames: 'basePreInscricaoVestibularFeteps', 'basePreInscritoVestibular' e 'base25'
# Isso combina as diferentes fontes de dados em um único DataFrame
base25 = pd.concat([basePreInscricaoVestibularFeteps, basePreInscritoVestibular, base25])


In [169]:
# Substitui os valores 'NULL' ou valores nulos na coluna 'processoSeletivoNome' com os valores correspondentes da coluna 'formaIngresso'
base25.loc[base25['processoSeletivoNome'].isin(['NULL', 'Null', None, '']), 'processoSeletivoNome'] = base25.loc[base25['processoSeletivoNome'].isin(['NULL', 'Null', None, '']), 'formaingresso']


In [170]:
#altere para Desafio Eseg as linhas na coluna processoSeletivoNome onde conterem a esafio em alguma parte do texto
base25.loc[base25['processoSeletivoNome'].str.contains('esafio', case=False), 'processoSeletivoNome'] = '25.1 - Desafio Eseg'

base25.loc[base25['processoSeletivoNome'].str.contains('transf', case=False), 'processoSeletivoNome'] = '25.1 - Transferência'

# Altera o processo seletivo PARA O FORMATO DE TITULO
base25['processoSeletivoNome'] = base25['processoSeletivoNome'].str.title()


In [171]:
# prompt: excluir os 8 primeiros caractere da coluna processoSeletivoNome quando o valor for diferente de sem processo definido

base25.loc[base25['processoSeletivoNome'] != 'Sem Processo Definido', 'processoSeletivoNome'] = base25.loc[base25['processoSeletivoNome'] != 'Sem Processo Definido', 'processoSeletivoNome'].str[7:]

# prompt: onde o processoSeletivoNome contem feteps, alterepara redação feteps

base25.loc[base25['processoSeletivoNome'].str.contains('FETEPS', case=False), 'processoSeletivoNome'] = 'Redação FETEPS'

base25.loc[base25['processoSeletivoNome'] == '', 'processoSeletivoNome'] = 'Sem Processo Definido'

In [172]:
# Resetando index
base25 = base25.reset_index(drop=True)

# criando verificação se é um matriculado
base25['Matriculado'] = ''
base25.loc[base25['etapaNome'].isin(['Matriculado', 'Matrícula Cancelada', 'Desistente']), 'Matriculado'] = 'Matriculado'

# Atribui o valor da coluna 'ultimaAlteracaoEtapa' para 'dataMatricula' na base25 onde a coluna 'Matriculado' é igual a 'Matriculado'
base25.loc[base25['Matriculado'] == 'Matriculado', 'dataMatricula'] = base25['ultimaAlteracaoEtapa']

# Converter a coluna 'dataMatricula' para datetime e, em seguida, extrair apenas a data
base25['dataMatricula'] = pd.to_datetime(base25['dataMatricula'], errors='coerce').dt.date

# Converter a coluna 'dataMatricula' para datetime
base24['dataMatricula'] = pd.to_datetime(base24['dataMatricula'], dayfirst=True).dt.date

In [173]:
# Trocando NULL por vazio
base25 = base25.replace('NULL', '')
base25 = base25.replace('Null', '')

In [174]:
# Função para separar a última palavra e o restante
def separar_ultima_palavra(texto):
    partes = texto.rsplit(' ', 1)
    if len(partes) == 2:
        restante, ultima_palavra = partes
    else:
        restante = texto
        ultima_palavra = ''
    return pd.Series([restante, ultima_palavra])

# Aplicar a função e criar novas colunas
base25[['cursoNome', 'turnoNome']] = base25['ofertaCursoNome'].apply(separar_ultima_palavra)

# Extrair parte da string a partir do 7º caractere
base25['cursoNome'] = base25['cursoNome'].str.slice(start=7)

In [175]:
# Alterando tipo da coluna
base25['momento'] = pd.to_datetime(base25['momento'])

# Criando variável com data da prova do Desafio ESEG
p_01 = pd.to_datetime('2024-09-29')
p_02 = pd.to_datetime('2024-11-24')
p_03 = pd.to_datetime('2024-12-08')
p_04 = pd.to_datetime('2025-01-15')

# Criando coluna de data da prova e aplicando a função para trazer a data da prova para qual o candidato de inscreveu
base25['dataProva'] = base25.apply(
                                  lambda row: (
                                      p_01 if (row['momento'] < p_01 and 'Desafio' in row['processoSeletivoNome']) else
                                      p_02 if (row['momento'] < p_02 and 'Desafio' in row['processoSeletivoNome']) else
                                      p_03 if (row['momento'] < p_03 and 'Desafio' in row['processoSeletivoNome']) else
                                      p_04 if (row['momento'] < p_04 and 'Desafio' in row['processoSeletivoNome']) else ''
                                  ), axis=1
)

# Alterando tipo da coluna
base25['dataProva'] = pd.to_datetime(base25['dataProva'], format='%Y-%m-%d')


In [176]:
base25['data'] = base25['momento'].dt.date
base24['data'] = base24['momento']

# Garantindo que todas as colunas de data estejam no formato datetime com o formato correto
base25['data'] = pd.to_datetime(base25['data'], format='%d/%m/%Y', dayfirst=True)
base24['data'] = pd.to_datetime(base24['data'], format='%d/%m/%Y', dayfirst=True)
base_aux['data24'] = pd.to_datetime(base_aux['data24'], format='%d/%m/%Y', dayfirst=True)
base_aux['data25'] = pd.to_datetime(base_aux['data25'], format='%d/%m/%Y', dayfirst=True)

In [177]:

# Trazendo a coluna de posição nas bases para comparativo
base25 = pd.merge(base25, base_aux[['data25','pos25']], left_on='data', right_on='data25', how='left')
base24 = pd.merge(base24, base_aux[['data24','pos24']], left_on='data', right_on='data24', how='left')

# Excluindo as colunas extras após o merge, se necessário
base25 = base25.rename(columns={'data25': 'dataInscricao25'})
base24 = base24.rename(columns={'data24': 'dataInscricao24'})

# Alterando nomes das colunas para posicao
base25 = base25.rename(columns={'pos25': 'posicao'})
base24 = base24.rename(columns={'pos24': 'posicao'})


In [178]:
# Criando coluna de posicaoHoje e dataHoje
base25['dataHoje'] = hoje
base24['dataHoje'] = hoje

base25['dataHoje'] = pd.to_datetime(base25['dataHoje'], format='%Y-%m-%d')
base24['dataHoje'] = pd.to_datetime(base24['dataHoje'], format='%Y-%m-%d')

base25 = pd.merge(base25, base_aux[['data25','pos25']], left_on='dataHoje', right_on='data25', how='left')
base24 = pd.merge(base24, base_aux[['data25','pos25']], left_on='dataHoje', right_on='data25', how='left')

# Alterando nomes das colunas para posicao
base25 = base25.rename(columns={'pos25': 'posicaoHoje'})
base24 = base24.rename(columns={'pos25': 'posicaoHoje'})

# Excluindo as colunas extras após o merge, se necessário
base25 = base25.drop(columns=['data25'])
base24 = base24.drop(columns=['data25'])

In [179]:
#Criando coluna de posição da matricula
base25['dataMatricula'] = pd.to_datetime(base25['dataMatricula'], format='%Y-%m-%d')
base24['dataMatricula'] = pd.to_datetime(base24['dataMatricula'], format='%Y-%m-%d')

base25 = pd.merge(base25, base_aux[['data25','pos25']], left_on='dataMatricula', right_on='data25', how='left')
base24 = pd.merge(base24, base_aux[['data24','pos24']], left_on='dataMatricula', right_on='data24', how='left')

# Alterando nomes das colunas para posicao
base25 = base25.rename(columns={'pos25': 'posicaoMatricula'})
base24 = base24.rename(columns={'pos24': 'posicaoMatricula'})

# Alterando nomes das colunas
base25 = base25.rename(columns={'data25': 'dataMatricula25'})
base24 = base24.rename(columns={'data24': 'dataMatricula24'})

In [180]:
# Definindo a posição final do intake (202 é o valor alvo)
finalIntake = 202

# Garantindo que a coluna 'posicaoHoje' seja do tipo numérico
base25['posicaoHoje'] = pd.to_numeric(base25['posicaoHoje'], errors='coerce')

# Capturando o valor máximo da coluna 'posicaoHoje', que representa a posição atual
posHoje = base25['posicaoHoje'].max()

# Calculando a quantidade de dias restantes para o término do intake e criando a coluna 'diasParaTerminoIntake'
# Subtrai a posição atual ('posicaoHoje') do valor final do intake
base25['diasParaTerminoIntake'] = finalIntake - posHoje


In [181]:
# Cria um conjunto com os e-mails da base_trv para facilitar a busca
emailsTRV = set(base_trv['E-mail'])

# Cria uma nova coluna 'preenchimentoTRV' vazia
base25['preenchimentoTRV'] = ''

# Preenche a coluna 'preenchimentoTRV' com 'TRV preenchido' quando o e-mail está presente em emailsTRV
base25.loc[base25['email'].isin(emailsTRV), 'preenchimentoTRV'] = 'TRV preenchido'


In [182]:
# Excluindo duplicados onde 'preenchimentoTRV' é 'TRV preenchido', mantendo a primeira ocorrência,
# exceto se a coluna 'etapaNome' for igual a 'Matriculado'
base25 = base25[~((base25['preenchimentoTRV'] == 'TRV preenchido') &
                  (base25['etapaNome'] != 'Matriculado') &
                  base25.duplicated(subset=['email'], keep='first'))]

# Alterando para sem processo definidoonde for '' na coluna pricessoSeletivoNome
base25.loc[base25['processoSeletivoNome'] == '', 'processoSeletivoNome'] = 'Sem Processo Definido'

In [183]:
# Corrigindo nulos
base25['processoSeletivoNome'] = base25['processoSeletivoNome'].fillna('Sem Processo Definido')
base25.loc[base25['processoSeletivoNome'] == '', 'processoSeletivoNome'] = 'Sem Processo Definido'
base25.loc[base25['processoNome'] == 'Inscrições e Matrículas', 'processoNome'] = 'Site'

In [184]:
# Define o valor da meta e cria a coluna 'Meta'
metaMatriculas = 390
base25['metaMatriculas'] = metaMatriculas

# Calcula o percentual da meta atingido com base no número de matriculados
base25['percentualDaMetaMatriculas'] = (base25[base25['Matriculado'] == 'Matriculado' ]['Matriculado'].count()) / (base25['metaMatriculas'].max())

# criando coluna de horário da atividade com base na coluna momento
base25['horario'] = base25['momento'].dt.strftime('%H:%M')

In [185]:
# Calcula a quantidade de matrículas faltantes para atingir a meta, subtraindo o número de matrículas já realizadas (com status 'Matriculado') da meta definida
matriculasFaltaParaMeta = metaMatriculas - (base25[base25['Matriculado'] == 'Matriculado']['Matriculado'].count())

# Calcula a quantidade de dias restantes até o término do intake, com base no valor máximo da coluna 'diasParaTerminoIntake'
diasParaterminoIntake = base25['diasParaTerminoIntake'].max()

# Calcula o número de matrículas necessárias por dia para atingir a meta dentro do prazo restante
matriculasFaltaPorDia = matriculasFaltaParaMeta / diasParaterminoIntake

# Adiciona as informações de matrículas faltantes para atingir a meta e matrículas necessárias por dia no DataFrame 'base25'
base25['matriculasParaMeta'] = matriculasFaltaParaMeta
base25['matriculasPorDia'] = matriculasFaltaPorDia

# Calcula o número de inscritos que já avançaram da etapa 'Pré-Inscrito'
inscritos = base25[base25['etapaNome'] != 'Pré-Inscrito']['etapaNome'].count()

# Define a meta de inscrições e calcula a quantidade de inscrições faltantes para atingir essa meta
metaInscricoes = 7500
inscricoesFaltaParaMeta = metaInscricoes - inscritos

# Adiciona as informações de inscrições faltantes para atingir a meta no DataFrame 'base25'
base25['inscricoesParaMeta'] = inscricoesFaltaParaMeta

# Calcula o número de inscrições necessárias por dia para atingir a meta de inscrições dentro do prazo restante
inscricoesFaltaPorDia = inscricoesFaltaParaMeta / diasParaterminoIntake
base25['inscricoesPorDia'] = inscricoesFaltaPorDia

# Calcula o percentual da meta atingido com base no número de inscrições
base25['percentualDaMetaInscricoes'] = (base25[base25['etapaNome'] != 'Pré-Inscrito']['etapaNome'].count()) / (metaInscricoes)


In [186]:
# base25['etapaNome'].value_counts()

In [187]:
# base24['etapaNome'].value_counts()

In [188]:
# Adicionadno intake nas bases
base25_comp = base25.copy()
base25_comp['Intake'] = '20251'
base24['Intake'] = '20241'

# Criando comparativo
comparativo = pd.concat([base25_comp, base24])

# Padronizando tipo da coluna
comparativo['processoSeletivoNome'] = comparativo['processoSeletivoNome'].str.title()

In [189]:
# Trazendo informaçõexs de campanhas tracking

baseTracking = abrir_plan('campanha_tracking_rubeus_script', 0)

baseTracking = baseTracking.replace('"', '')

# Replacing double quotes with empty strings in all columns of the DataFrame
for col in baseTracking.columns:
    if baseTracking[col].dtype == 'object':
        baseTracking[col] = baseTracking[col].str.replace('"', '')

baseTracking = baseTracking[~baseTracking['utmMediumCampanha'].str.contains('javascript', na=False)]

att_plan('campanha_tracking_rubeus_script', 'tratada', baseTracking)

baseTracking = baseTracking.rename(columns={'pessoaNome': 'nomePessoa'})

colunas = ['idDoContato',
    'idRegistro',
    # 'canalNome',
    'nomePessoa',
    'campanhaTracking',
    'origemCampanha',
    'tipoOrigemCampanha',
    'urlOrigemCampanha',
    'dispositivoCampanha',
    'utmSourceCampanha',
    'utmMediumCampanha',
    'utmContentCampanha'
]

# Ordena pelo momentoInteracao em ordem decrescente
baseTracking = baseTracking.sort_values(by='momentoInteracao', ascending=False)

baseTracking = baseTracking.drop_duplicates(subset='nomePessoa', keep='first')

baseInscritos = base25[base25['etapaNome'] != 'Pré-Inscrito']

basePreInscritos = base25[base25['etapaNome'] == 'Pré-Inscrito']

baseInscritos = pd.merge(baseInscritos, baseTracking[colunas], left_on='pessoaNome', right_on='nomePessoa', how='left')

base25Origens = base25.copy()

base25 = pd.concat([baseInscritos, basePreInscritos])


# base25 = base25.drop(columns=['canalNome_y'])
base25 = base25.drop(columns=['idRegistro_y'])
base25 = base25.rename(columns={'canalNome_x': 'canalNome', 'idRegistro_x': 'idRegistro'})

In [190]:
baseMat = base25[base25['etapaNome'] == 'Matriculado'].drop_duplicates(subset=['pessoaNome'])
baseNaoMat = base25[base25['etapaNome'] != 'Matriculado']
base25 = pd.concat([baseMat, baseNaoMat], ignore_index=True)

In [191]:
dados = {
    "pessoaNome": [
        "Julia Santiago Cuneo Danigno", "Ana Paula Ramos Da Silva",
        "Joao Pedro De Souza Paulino", "Aline De Lima Tavares",
        "Luiz Henrique Araujo Da Silva", "Beatriz Zoldan Gonçalves",
        "Pedro Henrique Amorim Santos", "Ester Caetano Santos Silva",
        "Lucas Hideki Mendes Tanaka", "Sarah Mariana Silva Ribeiro",
        "Keviny Felix pina", "Henrique Noé Diniz",
        "Albert Dos Santos Mariano", "Karen dos Anjos Hashimoto",
        "João Vitor Isicawa Gonçalves", "Luis Fernando dos Santos Araújo",
        "Pedro Charletto Pereira", "Gabriela França Da Rocha",
        "Paola Marinho dos Santos", "Henrique Pinheiro De Lima",
        "Giovanna Pesente", "Maria Eduarda Gomes Rodrigues Muniz",
        "Alexandra Silva Rocha", "Erick Henrique do Bom Jesus Ferreira"
    ],
    "campanhaTracking": [
        None, None, None, None, None, None, None, None, "CAC",
        "CAC", "CAC", "CAC", "CAC", None, None,
        "prospeccao_-_formas_de_ingresso_2025-1_-_envio_2_nao_abriram_e-mail_anterior",
        None, None, None, None, None, None, None, None
    ],
    "origemCampanha": [
        "Instagram", "Instagram", "Google", "Google",
        "Facebook", "TikTok", "Facebook", "Facebook",
        "Acesso direto", "Acesso direto", "Acesso direto", "Acesso direto",
        "Acesso direto", "Google", "Google", "Gmail",
        "Google", "Acesso direto", "Acesso direto", "Google",
        "Google", "Google", "Google", "Gmail"
    ],
    "tipoOrigemCampanha": [
        "Acesso direto", "Acesso direto", "Acesso direto", "Acesso direto",
        "Acesso direto", "Acesso direto", "Acesso direto", "Acesso direto",
        "Acesso direto", "Acesso direto", "Acesso direto", "Acesso direto",
        "Acesso direto", "Busca Orgânica", "Acesso direto", "Email",
        "Busca Orgânica", "Acesso direto", "Acesso direto", "Busca Orgânica",
        "Busca Orgânica", "Busca Orgânica", "Busca Orgânica", "Email"
    ],
    "urlOrigemCampanha": [
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None
    ],
    "dispositivoCampanha": [
        "Smartphone", "Computador", "Smartphone", "Computador",
        "Smartphone", "Computador", "Smartphone", "Computador",
        "Smartphone", "Computador", "Smartphone", "Computador",
        "Computador", "Computador", "Computador", "Smartphone",
        "Computador", "Computador", "Computador", "Computador",
        "Computador", "Computador", "Smartphone", "Smartphone"
    ],
    "utmSourceCampanha": [
        "CAC", "CAC", "CAC", "CAC", "CAC", "CAC", "CAC", "CAC",
        "CAC", "CAC", "CAC", "CAC", "CAC", None, None, None,
        None, None, None, None, None, None, None, None
    ],
    "utmMediumCampanha": [
        "CAC", "CAC", "CAC", "CAC", "CAC", "CAC", "CAC", "CAC",
        "CAC", "CAC", "CAC", "CAC", "CAC", None, None, None,
        None, None, None, None, None, None, None, None
    ]
}

# Criar um DataFrame a partir do dicionário
novos_dados = pd.DataFrame(dados)

# Realizar o merge (junção) com base na coluna 'Nome'
base25 = base25.merge(novos_dados, on='pessoaNome', how='left', suffixes=('', '_novo'))

# Atualizar os campos na base25 com os dados da nova base
for coluna in novos_dados.columns:
    if coluna != 'pessoaNome':
        base25[coluna].fillna(base25[f'{coluna}_novo'], inplace=True)

# Remover as colunas auxiliares criadas pelo merge
base25.drop(columns=[f'{coluna}_novo' for coluna in novos_dados.columns if coluna != 'pessoaNome'], inplace=True)

base25.drop(columns='nomePessoa', inplace=True)

base25 = base25.drop(columns=['idRegistro'])

<ipython-input-191-6f1c04b2782d>:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base25[coluna].fillna(base25[f'{coluna}_novo'], inplace=True)


In [192]:
att_plan('Monitoramento Inscrições e Matrículas - 25.1', 'base', base25)

In [193]:
dataHora = base25['att_em'].max()
dataHora = dataHora[:-3]
data = dataHora[:-6]
inscritos25 = len(base25[base25['processoNome'] == 'Site'])

comparativo['posicaoHoje'] = comparativo['posicaoHoje'].astype(float)
comparativo['posicao'] = comparativo['posicao'].astype(float)

inscritos24Hoje = len(comparativo[
    (comparativo['Intake'] == '20241') &
    (comparativo['posicao'] <= comparativo['posicaoHoje']) &
    (comparativo['processoNome'] != 'Google Form Leads')
])

crescimentoInscricao = (inscritos25 - inscritos24Hoje) / inscritos24Hoje
comparativo['crescimentoInscricoes'] = crescimentoInscricao
crescimentoInscricao = crescimentoInscricao * 100
crescimentoInscricao = f'{crescimentoInscricao:.2f}'

# data24 recebe a dataHora menos 364 dias
dataHora = pd.to_datetime(dataHora, dayfirst=True)
data24 = dataHora - pd.DateOffset(days=364)
data24 = data24.strftime('%d/%m/%Y')

dataHora = pd.to_datetime(dataHora, dayfirst=True)
dataHora = dataHora.strftime('%d/%m/%Y - %H:%M')

matriculados25 = len(comparativo[(comparativo['Intake'] == '20251') & (comparativo['Matriculado'] == 'Matriculado')])

comparativo['posicaoMatricula'] = comparativo['posicaoMatricula'].astype(float)
matriculados24Hoje = len(comparativo[
    (comparativo['Intake'] == '20241') &
    (comparativo['posicaoMatricula'] <= comparativo['posicaoHoje']) &
    (comparativo['Matriculado'] == 'Matriculado')
])

crescimentoMatriculas = (matriculados25 - matriculados24Hoje) / matriculados24Hoje
comparativo['crescimentoMatriculas'] = crescimentoMatriculas
crescimentoMatriculas = crescimentoMatriculas * 100
crescimentoMatriculas = f'{crescimentoMatriculas:.2f}'

comparativo['taxaConversao251'] = matriculados25 / inscritos25
comparativo['taxaConversao241'] = matriculados24Hoje / inscritos24Hoje

In [194]:
base25 = base25.reset_index(drop=True)
base25['posicaoHoje'][0]

124

In [195]:
base25['posicao'] = base25['posicao'].astype(int)
inscritos25Ontem = len(base25[base25['posicao'] == (base25['posicaoHoje'][0] - 1) ])
inscritos25SoHoje = len(base25[base25['posicao'] == base25['posicaoHoje'][0] ])

df24 = comparativo[(comparativo['Intake'] == '20241') & (comparativo['processoNome'] != 'Google Form Leads')]
df24['posicao'] = df24['posicao'].astype(int)
df24['posicaoHoje'] = df24['posicaoHoje'].astype(int)
inscritos24Ontem = len(df24[df24['posicao'] == (df24['posicaoHoje'][0] - 1) ])
inscritos24SoHoje = len(df24[df24['posicao'] == df24['posicaoHoje'][0] ])


crescimentoInscricaoSoHoje = ((inscritos25SoHoje - inscritos24SoHoje) / inscritos24SoHoje * 100)
crescimentoInscricaoSoHoje = f'{crescimentoInscricaoSoHoje:.2f}'

crescimentoInscricaoOntem = ((inscritos25Ontem - inscritos24Ontem) / inscritos24Ontem * 100)
crescimentoInscricaoOntem = f'{crescimentoInscricaoOntem:.2f}'

diaOntem25 = pd.to_datetime(dataHora, dayfirst=True) - pd.DateOffset(days=1)
diaOntem24 = diaOntem25 - pd.DateOffset(days=364)
diaOntem25 = diaOntem25.strftime('%d/%m/%Y')
diaOntem24 = diaOntem24.strftime('%d/%m/%Y')

In [196]:
att_plan('Monitoramento Inscrições e Matrículas - 25.1', 'comparativo', comparativo)

In [197]:
Site = len(base25[(base25['etapaNome'] != 'Pré-Inscrito') & (base25['baseDeOrigem'] == 'SITE')])
PreFeteps = len(base25[(base25['etapaNome'] != 'Pré-Inscrito') & (base25['baseDeOrigem'] == 'PRÉ-INSCRITO VESTIBULAR - FETEPS')])
PreSite = len(base25[(base25['etapaNome'] != 'Pré-Inscrito') & (base25['baseDeOrigem'] == 'PRÉ-INSCRITO VESTIBULAR - SITE')])
CadastroFeteps = len(base25[(base25['etapaNome'] != 'Pré-Inscrito') & (base25['baseDeOrigem'] == 'CADASTRO - FETEPS')])

In [198]:
relatorio  = f'''*MONITORAMENTO GRADUAÇÃO - {dataHora}*

*INSCRIÇÕES ONTEM*
INSCRITOS ONTEM 25.1 ({diaOntem25}) - {inscritos25Ontem}
INSCRITOS ONTEM 24.1 ({diaOntem24}) - {inscritos24Ontem}
CRESCIMENTO INSCRITOS ONTEM 25.1 x 24.1 - ({crescimentoInscricaoOntem}%)

*INSCRIÇÕES HOJE*
INSCRITOS DE HOJE 25.1 ({data}) - {inscritos25SoHoje}
INSCRITOS DE HOJE 24.1 ({data24}) - {inscritos24SoHoje}
CRESCIMENTO INSCRITOS DE HOJE 25.1 x 24.1 - ({crescimentoInscricaoSoHoje}%)

*INSCRIÇÕES TOTAIS*
TOTAL DE INSCRITOS 25.1 EM {data} - {inscritos25}
TOTAL DE INSCRITOS 24.1 EM {data24} - {inscritos24Hoje}
CRESCIMENTO PERCENTUAL INSCRITOS 25.1 x 24.1 - ({crescimentoInscricao}%)

*INSCRITOS POR FONTE*
SITE - {Site}
PRÉ-INSCRITO VESTIBULAR - SITE - {PreSite}
PRÉ-INSCRITO VESTIBULAR - FETEPS - {PreFeteps}
CADASTRO - FETEPS - {CadastroFeteps}

*MATRÍCULAS*
TOTAL DE MATRICULADOS 25.1 EM {data}  -  {matriculados25}
TOTAL DE MATRICULADOS 24.1 EM {data24}  -  {matriculados24Hoje}
CRESCIMENTO PERCENTUAL MATRICULADOS 25.1 x 24.1 - ({crescimentoMatriculas}%)

RELATÓRIO DISPONÍVEL EM - https://abre.ai/monitoramento-inscricoes-e-matriculas-251'''

In [199]:
# prompt: Transformando o relatorio em um arquivo txt

with open('relatorio.txt', 'w') as f:
  f.write(relatorio)

from google.colab import files
files.download('relatorio.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [201]:
import pandas_gbq
from pandas_gbq import to_gbq
from google.oauth2 import service_account

credencial = service_account.Credentials.from_service_account_file(r'/content/cloud-engineer-444301-r6-a1e2ea45a24e.json',
                                                                   scopes=['https://www.googleapis.com/auth/bigquery'])

In [202]:
base25_gbq = base25.copy()

table_schema_base25_gbq = [
    {"name": "momento", "type": "TIMESTAMP"},
    {"name": "minutosEtapa", "type": "STRING"},
    {"name": "ultimaAlteracaoEtapa", "type": "STRING"},
    {"name": "pessoa", "type": "STRING"},
    {"name": "pessoaNome", "type": "STRING"},
    {"name": "agendamento", "type": "STRING"},
    {"name": "agendamentoNome", "type": "STRING"},
    {"name": "ofertaCursoNome", "type": "STRING"},
    {"name": "processoSeletivoNome", "type": "STRING"},
    {"name": "unidadeNome", "type": "STRING"},
    {"name": "modalidadeNome", "type": "STRING"},
    {"name": "etapaNome", "type": "STRING"},
    {"name": "statusNome", "type": "STRING"},
    {"name": "responsavelNome", "type": "STRING"},
    {"name": "localOfertaNome", "type": "STRING"},
    {"name": "dias", "type": "STRING"},
    {"name": "objecaoNome", "type": "STRING"},
    {"name": "concorrenteNome", "type": "STRING"},
    {"name": "razaoOportunidadeNome", "type": "STRING"},
    {"name": "notaEnem", "type": "STRING"},
    {"name": "compareceuAtividade", "type": "STRING"},
    {"name": "dataProva", "type": "TIMESTAMP"},
    {"name": "valorCurso", "type": "STRING"},
    {"name": "observacaoPerda", "type": "STRING"},
    {"name": "endereco", "type": "STRING"},
    {"name": "numero", "type": "STRING"},
    {"name": "bairro", "type": "STRING"},
    {"name": "cep", "type": "STRING"},
    {"name": "email", "type": "STRING"},
    {"name": "telefone", "type": "STRING"},
    {"name": "cpf", "type": "STRING"},
    {"name": "datanascimento", "type": "STRING"},
    {"name": "sexo", "type": "STRING"},
    {"name": "cidadeEstado", "type": "STRING"},
    {"name": "canalNome", "type": "STRING"},
    {"name": "processoNome", "type": "STRING"},
    {"name": "cursoNome", "type": "STRING"},
    {"name": "codigoPessoa", "type": "STRING"},
    {"name": "codigoOfertaCurso", "type": "STRING"},
    {"name": "momentoGanho", "type": "STRING"},
    {"name": "statusOportunidade", "type": "STRING"},
    {"name": "escolaDeOrigem", "type": "STRING"},
    {"name": "empresaConveniada", "type": "STRING"},
    {"name": "formaingresso", "type": "STRING"},
    {"name": "tags", "type": "STRING"},
    {"name": "cupomDeDesconto", "type": "STRING"},
    {"name": "anoDeEstudoETEC", "type": "STRING"},
    {"name": "temperatura", "type": "STRING"},
    {"name": "att_em", "type": "STRING"},
    {"name": "baseDeOrigem", "type": "STRING"},
    {"name": "Matriculado", "type": "STRING"},
    {"name": "dataMatricula", "type": "TIMESTAMP"},
    {"name": "turnoNome", "type": "STRING"},
    {"name": "data", "type": "TIMESTAMP"},
    {"name": "dataInscricao25", "type": "TIMESTAMP"},
    {"name": "posicao", "type": "INTEGER"},
    {"name": "dataHoje", "type": "TIMESTAMP"},
    {"name": "posicaoHoje", "type": "INTEGER"},
    {"name": "dataMatricula25", "type": "TIMESTAMP"},
    {"name": "posicaoMatricula", "type": "STRING"},
    {"name": "diasParaTerminoIntake", "type": "INTEGER"},
    {"name": "preenchimentoTRV", "type": "STRING"},
    {"name": "metaMatriculas", "type": "INTEGER"},
    {"name": "percentualDaMetaMatriculas", "type": "FLOAT"},
    {"name": "horario", "type": "STRING"},
    {"name": "matriculasParaMeta", "type": "INTEGER"},
    {"name": "matriculasPorDia", "type": "FLOAT"},
    {"name": "inscricoesParaMeta", "type": "INTEGER"},
    {"name": "inscricoesPorDia", "type": "FLOAT"},
    {"name": "percentualDaMetaInscricoes", "type": "FLOAT"},
    {"name": "idDoContato", "type": "STRING"},
    {"name": "campanhaTracking", "type": "STRING"},
    {"name": "origemCampanha", "type": "STRING"},
    {"name": "tipoOrigemCampanha", "type": "STRING"},
    {"name": "urlOrigemCampanha", "type": "STRING"},
    {"name": "dispositivoCampanha", "type": "STRING"},
    {"name": "utmSourceCampanha", "type": "STRING"},
    {"name": "utmMediumCampanha", "type": "STRING"},
    {"name": "utmContentCampanha", "type": "STRING"}
]



# Carregar para o BigQuery com o esquema especificado
to_gbq(
    dataframe=base25_gbq,  # Substitua pelo seu DataFrame
    destination_table="teste02_sp_dw.teste02_base",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",  # Opções: "fail", "replace", "append"
    table_schema=table_schema_base25_gbq,  # Especifica o esquema da tabela
    credentials=credencial,  # Sua credencial configurada
)


100%|██████████| 1/1 [00:00<00:00, 1664.41it/s]


In [203]:
baseTracking_gbq = baseTracking.copy()

table_schema_baseTracking_gbq = [
    {"name": "idDoContato", "type": "STRING"},
    {"name": "momentoInteracao", "type": "STRING"},
    {"name": "idRegistro", "type": "STRING"},
    {"name": "momentoCriacaoRegistro", "type": "STRING"},
    {"name": "minutosEtapa", "type": "STRING"},
    {"name": "ultimaAlteracaoEtapa", "type": "STRING"},
    {"name": "formaIngresso", "type": "STRING"},
    {"name": "nomePessoa", "type": "STRING"},
    {"name": "pessoaCodigo", "type": "STRING"},
    {"name": "agendamento", "type": "STRING"},
    {"name": "agendamentoNome", "type": "STRING"},
    {"name": "ofertaCursoNome", "type": "STRING"},
    {"name": "processoSeletivoNome", "type": "STRING"},
    {"name": "unidadeNome", "type": "STRING"},
    {"name": "modalidadeNome", "type": "STRING"},
    {"name": "etapaNome", "type": "STRING"},
    {"name": "statusNome", "type": "STRING"},
    {"name": "responsavelNome", "type": "STRING"},
    {"name": "localOfertaNome", "type": "STRING"},
    {"name": "dias", "type": "STRING"},
    {"name": "objecaoNome", "type": "STRING"},
    {"name": "concorrenteNome", "type": "STRING"},
    {"name": "razaoOportunidadeNome", "type": "STRING"},
    {"name": "dataProva", "type": "STRING"},
    {"name": "valorCurso", "type": "STRING"},
    {"name": "observacaoPerda", "type": "STRING"},
    {"name": "canalNome", "type": "STRING"},
    {"name": "processoNome", "type": "STRING"},
    {"name": "campanhaTracking", "type": "STRING"},
    {"name": "origemCampanha", "type": "STRING"},
    {"name": "tipoOrigemCampanha", "type": "STRING"},
    {"name": "urlOrigemCampanha", "type": "STRING"},
    {"name": "dispositivoCampanha", "type": "STRING"},
    {"name": "utmSourceCampanha", "type": "STRING"},
    {"name": "utmMediumCampanha", "type": "STRING"},
    {"name": "utmContentCampanha", "type": "STRING"},
    {"name": "utmCampaign", "type": "STRING"},
    {"name": "utmTerm", "type": "STRING"},
    {"name": "momentoUltimoGanho", "type": "STRING"}
]



# Carregar para o BigQuery com o esquema especificado
to_gbq(
    dataframe=baseTracking_gbq,  # Substitua pelo seu DataFrame
    destination_table="teste02_sp_dw.teste02_baseTrackingTratada",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",  # Opções: "fail", "replace", "append"
    table_schema=table_schema_baseTracking_gbq,  # Especifica o esquema da tabela
    credentials=credencial,  # Sua credencial configurada
)


100%|██████████| 1/1 [00:00<00:00, 7825.19it/s]


In [209]:
comparativo.momento = pd.to_datetime(comparativo.momento)

baseComparativo_gbq = comparativo.copy()

table_schema_baseComparativo_gbq = [
    {"name": "momento", "type": "DATETIME"},
    {"name": "minutosEtapa", "type": "STRING"},
    {"name": "ultimaAlteracaoEtapa", "type": "STRING"},
    {"name": "pessoa", "type": "STRING"},
    {"name": "pessoaNome", "type": "STRING"},
    {"name": "agendamento", "type": "STRING"},
    {"name": "agendamentoNome", "type": "STRING"},
    {"name": "ofertaCursoNome", "type": "STRING"},
    {"name": "processoSeletivoNome", "type": "STRING"},
    {"name": "unidadeNome", "type": "STRING"},
    {"name": "modalidadeNome", "type": "STRING"},
    {"name": "etapaNome", "type": "STRING"},
    {"name": "statusNome", "type": "STRING"},
    {"name": "responsavelNome", "type": "STRING"},
    {"name": "localOfertaNome", "type": "STRING"},
    {"name": "dias", "type": "STRING"},
    {"name": "objecaoNome", "type": "STRING"},
    {"name": "concorrenteNome", "type": "STRING"},
    {"name": "razaoOportunidadeNome", "type": "STRING"},
    {"name": "notaEnem", "type": "STRING"},
    {"name": "compareceuAtividade", "type": "STRING"},
    {"name": "dataProva", "type": "DATETIME"},
    {"name": "valorCurso", "type": "STRING"},
    {"name": "observacaoPerda", "type": "STRING"},
    {"name": "endereco", "type": "STRING"},
    {"name": "numero", "type": "STRING"},
    {"name": "bairro", "type": "STRING"},
    {"name": "cep", "type": "STRING"},
    {"name": "email", "type": "STRING"},
    {"name": "telefone", "type": "STRING"},
    {"name": "cpf", "type": "STRING"},
    {"name": "datanascimento", "type": "STRING"},
    {"name": "sexo", "type": "STRING"},
    {"name": "cidadeEstado", "type": "STRING"},
    {"name": "canalNome", "type": "STRING"},
    {"name": "processoNome", "type": "STRING"},
    {"name": "cursoNome", "type": "STRING"},
    {"name": "idRegistro", "type": "STRING"},
    {"name": "codigoPessoa", "type": "STRING"},
    {"name": "codigoOfertaCurso", "type": "STRING"},
    {"name": "momentoGanho", "type": "STRING"},
    {"name": "statusOportunidade", "type": "STRING"},
    {"name": "escolaDeOrigem", "type": "STRING"},
    {"name": "empresaConveniada", "type": "STRING"},
    {"name": "formaingresso", "type": "STRING"},
    {"name": "tags", "type": "STRING"},
    {"name": "cupomDeDesconto", "type": "STRING"},
    {"name": "anoDeEstudoETEC", "type": "STRING"},
    {"name": "temperatura", "type": "STRING"},
    {"name": "att_em", "type": "STRING"},
    {"name": "baseDeOrigem", "type": "STRING"},
    {"name": "Matriculado", "type": "STRING"},
    {"name": "dataMatricula", "type": "DATETIME"},
    {"name": "turnoNome", "type": "STRING"},
    {"name": "data", "type": "DATETIME"},
    {"name": "dataInscricao25", "type": "DATETIME"},
    {"name": "posicao", "type": "FLOAT"},
    {"name": "dataHoje", "type": "DATETIME"},
    {"name": "posicaoHoje", "type": "FLOAT"},
    {"name": "dataMatricula25", "type": "DATETIME"},
    {"name": "posicaoMatricula", "type": "FLOAT"},
    {"name": "diasParaTerminoIntake", "type": "FLOAT"},
    {"name": "preenchimentoTRV", "type": "STRING"},
    {"name": "metaMatriculas", "type": "FLOAT"},
    {"name": "percentualDaMetaMatriculas", "type": "FLOAT"},
    {"name": "horario", "type": "STRING"},
    {"name": "matriculasParaMeta", "type": "FLOAT"},
    {"name": "matriculasPorDia", "type": "FLOAT"},
    {"name": "inscricoesParaMeta", "type": "FLOAT"},
    {"name": "inscricoesPorDia", "type": "FLOAT"},
    {"name": "percentualDaMetaInscricoes", "type": "FLOAT"},
    {"name": "Intake", "type": "STRING"},
    {"name": "dataInscricao24", "type": "DATETIME"},
    {"name": "dataMatricula24", "type": "DATETIME"},
    {"name": "crescimentoInscricoes", "type": "FLOAT"},
    {"name": "crescimentoMatriculas", "type": "FLOAT"},
    {"name": "taxaConversao251", "type": "FLOAT"},
    {"name": "taxaConversao241", "type": "FLOAT"}
]



# Carregar para o BigQuery com o esquema especificado
to_gbq(
    dataframe=baseComparativo_gbq,  # Substitua pelo seu DataFrame
    destination_table="teste02_sp_dw.teste02_baseComparativo",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",  # Opções: "fail", "replace", "append"
    table_schema=table_schema_baseComparativo_gbq,  # Especifica o esquema da tabela
    credentials=credencial,  # Sua credencial configurada
)


100%|██████████| 1/1 [00:00<00:00, 7667.83it/s]
